In [1]:
import json
import random
import numpy as np
import pandas as pd
from csv import DictWriter
from deap import base, creator, tools

In [2]:
json_handle = open(r'../results/Customized_Data_2', 'r', encoding = 'utf-8')
json_string = json_handle.read()
instance = json.loads(json_string)
demand_ls = []
pop_ls = []
start_index = 1

for i in range(1, 101):
    demand = instance[f'customer_{i}']['demand']
    demand_ls.append(demand)

for demand in demand_ls:
    pop_ls.extend([start_index] * int(demand))
    start_index += 1

#print(pop_ls)

In [3]:
def ind2route(individual, instance): 
    '''decoding'''
    route = []
    depart_due_time = instance['depart']['due_time']
    max_vehicle_number = instance['max_vehicle_number']
    vehicle_number = 1
    initial_customer_demand = list(map(list, zip(individual, [1] * len(individual))))
    for vehicle_number in range(1, max_vehicle_number + 1):
        sub_route = [f"Vechile_{vehicle_number}"]
        elapsed_time = 0
        for customer_demand in initial_customer_demand:
            
            # Update elapsed time
            due_time = instance[f'customer_{customer_demand[0]}']['due_time']
            service_time = instance[f'customer_{customer_demand[0]}']['service_time']
            return_time = instance['distance_matrix'][0][customer_demand[0]]
            updated_elapsed_time = elapsed_time + service_time + 2*return_time
            
            # Validate vehicle load and elapsed time
            if customer_demand[1] == 1 and (updated_elapsed_time <= depart_due_time) and \
                (updated_elapsed_time - return_time < due_time):
                
            # Add to current sub-route
                sub_route.append(customer_demand[0])
                elapsed_time = updated_elapsed_time
                customer_demand[1] += -1
        if sub_route[1:] != []:
            route.append(sub_route)
    return route

In [4]:
def print_route(route, merge=False): 
    '''print route'''
    route_str = '0'
    sub_route_count = 0
    for sub_route in route:
        sub_route_str = '0'
        for customer_id in sub_route[1:]:
            sub_route_str = f'{sub_route_str} - {customer_id} - 0'
            route_str = f'{route_str} - {customer_id} - 0'
        if not merge:
            print(f' {sub_route[0]}\'s route: {sub_route_str}')
        route_str = f'{route_str} - 0'
        sub_route_count += 1
    if merge:
        print(route_str)

In [5]:
def eval_vrptw(individual, instance, unit_cost=1.0, init_cost=0, wait_cost=0): 
    '''fitness function'''
    
    route = ind2route(individual, instance)
    total_unsaving_demand = len(pop_ls)
    total_cost = 0
    time_cost = 0
    unsaving_cost = 0  
    
    for sub_route in route:
        elapsed_time = 0
        
        for customer_id in sub_route[1:]:
            
            ready_time = instance[f'customer_{customer_id}']['ready_time']
            service_time = instance[f'customer_{customer_id}']['service_time']
            distance = instance['distance_matrix'][0][customer_id]
            
            # Calculate time cost
            arrival_time = elapsed_time + distance
            time_cost += wait_cost * max(ready_time - arrival_time, 0)
            
            # Update elapsed time
            elapsed_time = arrival_time + service_time + distance
            
        # Calculate unsaving demand
        total_unsaving_demand += - len(sub_route[1:])
        
    # Calculate unsaving cost
    unsaving_cost = init_cost + unit_cost * total_unsaving_demand

    # Update total cost
    total_cost = unsaving_cost + time_cost
        
    fitness = 1.0 / total_cost * 10**6
    return (fitness, )

In [6]:
def mut_inverse_indexes(individual): 
    '''mutation rules'''
    start, stop = sorted(random.sample(range(len(individual)), 2))
    temp = individual[start:stop+1]
    temp.reverse()
    individual[start:stop+1] = temp
    return (individual, )

In [7]:
def run_gavrptw(instance, unit_cost, init_cost, wait_cost, ind_size, pop_size, \
    cx_pb, mut_pb, n_gen, export_csv=False):
    
    
    #define types
    creator.create('FitnessMax', base.Fitness, weights=(1.0, ))
    creator.create('Individual', list, fitness=creator.FitnessMax)

    #initialization
    toolbox = base.Toolbox()
    toolbox.register('indexes', random.sample, pop_ls, ind_size)
    toolbox.register('individual', tools.initIterate, creator.Individual, toolbox.indexes)
    toolbox.register('population', tools.initRepeat, list, toolbox.individual)

    #operatiors
    toolbox.register('evaluate', eval_vrptw, instance=instance, unit_cost=unit_cost, \
                      init_cost=init_cost, wait_cost=wait_cost)
    toolbox.register('select', tools.selRoulette)
    toolbox.register('mate', tools.cxTwoPoint)
    toolbox.register('mutate', mut_inverse_indexes)
    
    pop = toolbox.population(n=pop_size)

    # Evaluate the entire population
    fitnesses = list(map(toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit

    for g in range(n_gen):
        # Select the next generation individuals
        offspring = toolbox.select(pop, len(pop))
        # Clone the selected individuals
        offspring = list(map(toolbox.clone, offspring))

        # Apply crossover and mutation on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < cx_pb:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:
            if random.random() < mut_pb:
                toolbox.mutate(mutant)
                del mutant.fitness.values

        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = list(map(toolbox.evaluate, invalid_ind))
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit
            
        print(f' Evaluated {len(invalid_ind)} individuals')
        
        # The population is entirely replaced by the offspring
        pop[:] = offspring

        # Gather all the fitnesses in one list and print the stats
        fits = [ind.fitness.values[0] for ind in pop]
        length = len(pop)
        mean = sum(fits) / length
        sum2 = sum([x**2 for x in fits])
        std = abs(sum2 / length - mean**2)**0.5
        print(f'  Min {round(min(fits), 6)}')
        print(f'  Max {round(max(fits), 6)}')
        print(f'  Avg {round(mean, 6)}')
        print(f'  Std {round(std, 6)}')
        
    print('-- End of (successful) evolution --')
    
    best_ind = tools.selBest(pop, 1)[0]
    print(f'the best individual: {best_ind}')
    print(f'Fitness: {round(best_ind.fitness.values[0], 6)}')
    print_route(ind2route(best_ind, instance))
    print(f'Total cost: {1 / best_ind.fitness.values[0] * 10**6}')
   
        
    return pop

In [8]:
def main():
    '''main()'''
    random.seed(2022)

    instance = json.loads(json_string)

    unit_cost = 500.0
    init_cost = 0.0
    wait_cost = 10

    ind_size = 300
    pop_size = 100
    cx_pb = 0.9
    mut_pb = 0.1
    n_gen = 100

    export_csv = True

    run_gavrptw(instance=instance, unit_cost=unit_cost, init_cost=init_cost, \
        wait_cost=wait_cost, ind_size=ind_size, pop_size=pop_size, \
        cx_pb=cx_pb, mut_pb=mut_pb, n_gen=n_gen, export_csv=export_csv)


if __name__ == '__main__':
    main()

 Evaluated 90 individuals
  Min 1.06071
  Max 1.08692
  Avg 1.074338
  Std 0.006242
 Evaluated 95 individuals
  Min 1.057055
  Max 1.08764
  Avg 1.074862
  Std 0.006485
 Evaluated 91 individuals
  Min 1.055441
  Max 1.08764
  Avg 1.074921
  Std 0.006816
 Evaluated 83 individuals
  Min 1.05567
  Max 1.08723
  Avg 1.075615
  Std 0.006608
 Evaluated 88 individuals
  Min 1.05567
  Max 1.090707
  Avg 1.075997
  Std 0.007378
 Evaluated 92 individuals
  Min 1.05567
  Max 1.090714
  Avg 1.07627
  Std 0.007343
 Evaluated 92 individuals
  Min 1.058264
  Max 1.091359
  Avg 1.075818
  Std 0.007979
 Evaluated 94 individuals
  Min 1.058264
  Max 1.090001
  Avg 1.076138
  Std 0.007834
 Evaluated 94 individuals
  Min 1.058143
  Max 1.090001
  Avg 1.07627
  Std 0.008145
 Evaluated 88 individuals
  Min 1.059044
  Max 1.089679
  Avg 1.076929
  Std 0.008749
 Evaluated 83 individuals
  Min 1.059149
  Max 1.089679
  Avg 1.076216
  Std 0.008722
 Evaluated 90 individuals
  Min 1.060493
  Max 1.089758
  Avg 1.

 Evaluated 91 individuals
  Min 1.063885
  Max 1.087895
  Avg 1.077695
  Std 0.006455
 Evaluated 87 individuals
  Min 1.06466
  Max 1.091629
  Avg 1.077572
  Std 0.006281
 Evaluated 93 individuals
  Min 1.063322
  Max 1.091636
  Avg 1.077106
  Std 0.006539
 Evaluated 92 individuals
  Min 1.062067
  Max 1.09089
  Avg 1.076482
  Std 0.006259
-- End of (successful) evolution --
the best individual: [90, 13, 3, 65, 56, 87, 76, 20, 46, 15, 17, 98, 56, 70, 88, 16, 73, 45, 46, 10, 54, 31, 77, 56, 84, 16, 88, 16, 73, 64, 60, 85, 100, 29, 33, 92, 63, 25, 16, 77, 57, 93, 47, 40, 31, 38, 39, 74, 82, 34, 70, 97, 25, 57, 30, 59, 71, 2, 96, 7, 82, 78, 85, 57, 1, 6, 84, 81, 17, 74, 17, 7, 59, 13, 18, 91, 12, 76, 94, 88, 16, 84, 56, 77, 31, 54, 2, 46, 45, 52, 58, 35, 31, 76, 57, 74, 82, 25, 33, 98, 85, 51, 49, 15, 2, 12, 88, 42, 60, 17, 46, 15, 33, 81, 16, 63, 5, 75, 57, 33, 88, 19, 98, 74, 57, 53, 70, 55, 15, 68, 44, 40, 13, 87, 85, 28, 16, 58, 52, 45, 46, 2, 54, 18, 13, 35, 86, 7, 63, 34, 40, 47, 15